In [ ]:
cd /home/yuchen/pulse2percept

In [ ]:
import shapes
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pulse2percept as p2p
import pandas as pd
import skimage
from skimage.measure import label, regionprops, regionprops_table
import math
import string
from statistics import mean
from sklearn.linear_model import LinearRegression
import shapely.geometry as geom
import cv2 
from sklearn.linear_model import LinearRegression
import shapely.geometry as geom
import statsmodels.api as sm 
import pingouin as pg
from statsmodels.stats.outliers_influence import variance_inflation_factor
from matplotlib.ticker import FormatStrFormatter, MaxNLocator, MultipleLocator

In [ ]:
# calculate shapes 
data = shapes.load_shapes("/home/yuchen/shapes/data/shapes.h5", subjects=['12-005','51-009','52-001'],stim_class=None)

result = pd.DataFrame({})
for i in range(len(data)):
    label_img = skimage.measure.label(data['image'][i]>0)
    regions = regionprops(label_img)
    props = regionprops_table(label_img, properties=('centroid',
                                                     'orientation',
                                                     'major_axis_length',
                                                     'minor_axis_length',
                                                     'area',
                                                    'eccentricity',
                                                    'perimeter'))
    df = pd.DataFrame(props).astype('object')
    df.at[0,'centroid-0'] = df.iloc[:, 0].tolist()  # store centroid-x
    df.at[0,'centroid-1'] = df.iloc[:, 1].tolist()  # store centroid-y
    df.at[0,'orientation'] = df.iloc[:, 2].tolist()  # store orientation
    df.at[0,'major_axis_length'] = df.iloc[:, 3].tolist()  # major
    df.at[0,'minor_axis_length'] = df.iloc[:, 4].tolist()  # minor
    df.at[0,'area'] = df.iloc[:, 5].tolist()  # area
    df.at[0,'eccentricity'] = df.iloc[:, 6].tolist()  
    df.at[0,'perimeter'] = df.iloc[:, 7].tolist()  
    result = pd.concat([result, df.iloc[:1,:]],axis=0)

result = result.rename(columns={"area":"size", "orientation":"orientation_new", "eccentricity":"eccentricity_new" })
data = pd.concat([data,result.reset_index(drop=True)],axis=1)
data_copy = data.copy()

In [ ]:
count_0 = 0
count_1 = 0
test = []
lst = []
for i in range(len(data)):
    if data.electrode2[i] == str():
        lst.append('SingleElectrode')
    else:
        lst.append(data.stim_class[i])
data['stim_class'] = lst

cate = data[['subject','amp1','freq','electrode1','electrode2','stim_class']].drop_duplicates().reset_index(drop=True)

img = []
for i in range(len(cate)):
    df = data[(data['subject'] == cate['subject'][i]) & 
              (data['amp1'] == cate['amp1'][i]) & 
              (data['electrode1'] == cate['electrode1'][i]) &
              (data['electrode2'] == cate['electrode2'][i]) & 
              (data['stim_class'] == cate['stim_class'][i])
             ].reset_index(drop=True)
    if df.empty:
        img.append(np.zeros((384, 512)))
    else:
        if df.electrode2[0] == str():  # single electrode 
            if (len(df[df['num_regions']>1])/(len(df))) <= 0.1:
                count_1 += 1
                df_1 = df[df['num_regions'] == 1].reset_index(drop=True)

                df_centroid_0_avg = 0
                df_centroid_1_avg = 0
                for item in range(len(df_1)):
                    df_centroid_0_avg = df_centroid_0_avg + df_1['centroid-0'][item][0]
                    df_centroid_1_avg = df_centroid_1_avg + df_1['centroid-1'][item][0]
                df_centroid_0_avg = df_centroid_0_avg/len(df_1)
                df_centroid_1_avg = df_centroid_1_avg/len(df_1)

                # for drawings with more than one phosphenes, select the one with centroid cloest to the averaged centroid
                df_2 = df[df['num_regions'] > 1].reset_index(drop=True)
                if len(df_2) != 0:
                    for double_number in range(len(df_2)):
                        phosphene_id = 0
                        distance = 9999999
                        for double_item in range(len(df_2['centroid-0'][double_number])):
                            distance_temp = (df_2['centroid-0'][double_number][double_item] - df_centroid_0_avg)**2 + (df_2['centroid-1'][double_number][double_item] - df_centroid_1_avg)**2
                            if distance_temp < distance:
                                distance = distance_temp
                                phosphene_id = double_item
                        df_2.at[double_number,'centroid-0'] = [df_2['centroid-0'][double_number][phosphene_id]]
                        df_2.at[double_number,'centroid-1'] = [df_2['centroid-1'][double_number][phosphene_id]]

                df = pd.concat([df_1,df_2], axis=0).reset_index(drop=True)
                image_temp = np.zeros((len(df['image'][0]),len(df['image'][0][0])))
                for image in range(len(df)):
                    image_temp = image_temp + p2p.utils.images.shift_image(df['image'][image],df_centroid_1_avg-df['centroid-1'][image][0], df_centroid_0_avg-df['centroid-0'][image][0])
                image_temp = image_temp / len(df)
                img.append(image_temp)

            else:  
                phosphene1 = np.mean([p2p.utils.images.center_image(skimage.measure.label(image) == 1) for image in df.image], axis=0)
                phosphene2 = np.mean([p2p.utils.images.center_image(skimage.measure.label(image) == 2) for image in df.image], axis=0)
                phosphene1_centroid = np.mean([regionprops(skimage.measure.label(image>0))[0].centroid for image in df.image], axis=0)
                if sum(sum(phosphene2)) != 0:
                    count = (0,0)
                    phosphene2_centroid = 0
                    for image in df.image:
                        if len(regionprops(skimage.measure.label(image>0))) >= 2:
                            count = (count[0] + regionprops(skimage.measure.label(image>0))[1].centroid[0],count[1] + regionprops(skimage.measure.label(image>0))[1].centroid[1]) 
                            phosphene2_centroid += 1
                    phosphene2_centroid = (count[0] / phosphene2_centroid,count[1] / phosphene2_centroid)
                y_center, x_center = phosphene1.shape[0] / 2, phosphene1.shape[1] / 2
                phosphene1 = p2p.utils.images.shift_image(phosphene1, phosphene1_centroid[1] - x_center, phosphene1_centroid[0] - y_center)
                if sum(sum(phosphene2)) != 0:
                    phosphene2 = p2p.utils.images.shift_image(phosphene2, phosphene2_centroid[1] - x_center, phosphene2_centroid[0] - y_center)
                    image_temp = phosphene1 + phosphene2
                else:
                    image_temp = phosphene1
                test.append(image_temp)
                img.append(image_temp)
        elif df.electrode2[0] != str():  # double electrode
            phosphene1 = np.mean([p2p.utils.images.center_image(skimage.measure.label(image) == 1) for image in df.image], axis=0)
            phosphene2 = np.mean([p2p.utils.images.center_image(skimage.measure.label(image) == 2) for image in df.image], axis=0)
            phosphene1_centroid = np.mean([regionprops(skimage.measure.label(image>0))[0].centroid for image in df.image], axis=0)
            if sum(sum(phosphene2)) != 0:
                count = (0,0)
                phosphene2_centroid = 0
                for image in df.image:
                    if len(regionprops(skimage.measure.label(image>0))) >= 2:
                        count = (count[0] + regionprops(skimage.measure.label(image>0))[1].centroid[0],count[1] + regionprops(skimage.measure.label(image>0))[1].centroid[1]) 
                        phosphene2_centroid += 1
                phosphene2_centroid = (count[0] / phosphene2_centroid,count[1] / phosphene2_centroid)
            y_center, x_center = phosphene1.shape[0] / 2, phosphene1.shape[1] / 2
            phosphene1 = p2p.utils.images.shift_image(phosphene1, phosphene1_centroid[1] - x_center, phosphene1_centroid[0] - y_center)
            if sum(sum(phosphene2)) != 0:
                phosphene2 = p2p.utils.images.shift_image(phosphene2, phosphene2_centroid[1] - x_center, phosphene2_centroid[0] - y_center)
                image_temp = phosphene1 + phosphene2
            else:
                image_temp = phosphene1
            test.append(image_temp)
            img.append(image_temp)
            
cate['avg_image'] = img

In [ ]:
cate = cate[cate.stim_class != 'SpatialSummation'].reset_index(drop=True)
cate

In [ ]:
# eea
double = cate[cate.electrode2 != str()].reset_index(drop=True)
single = cate[cate.electrode2 == str()].reset_index(drop=True)

scale_dict = {0: 1.5, 1:1.5, 2:1.2,
             3:1, 4:1, 5:1,
             6:1.2, 7:1.2, 8:1,}

fig, axs= plt.subplots(ncols=3, nrows = 9, figsize=(21, 40))
fig.tight_layout()

row = 0

for i in [ 24, 31, 41, 57, 79 , 85, 93, 102, 108]:
    
        e1 = double.electrode1[i]
        e2 = double.electrode2[i]
        freq = double.freq[i]
        amp1 = double.amp1[i]
        subject = double.subject[i]
        single1 = single[(single.electrode1 == e1) & (single.freq == freq) &
                                (single.amp1 == amp1) & (single.subject == subject)]
        single2 = single[(single.electrode1 == e2) & (single.freq == freq) &
                                (single.amp1 == amp1) & (single.subject == subject)]
        
        new_height = int(round(single1.avg_image.iloc[0].shape[0]/scale_dict[row],0))
        new_width = int(round(single1.avg_image.iloc[0].shape[1]/scale_dict[row],0))
        axs[row,0].imshow(cv2.resize(single1.avg_image.iloc[0], (new_width, new_height)), cmap = 'gray_r')
        axs[row,1].imshow(cv2.resize(single2.avg_image.iloc[0], (new_width, new_height)), cmap = 'gray_r')       
        axs[row,2].imshow(cv2.resize(double.avg_image[i], (new_width, new_height)), cmap = 'gray_r')
        
        axs[row, 0].text(0.98, 0.02, str(e1), transform=axs[row, 0].transAxes,
                     ha='right', va='bottom', color='black', fontsize=64)
        axs[row, 1].text(0.98, 0.02, str(e2), transform=axs[row, 1].transAxes,
                         ha='right', va='bottom', color='black', fontsize=64)
        axs[row, 2].text(0.98, 0.02, str(e1) + '+' + str(e2), transform=axs[row, 2].transAxes,
                         ha='right', va='bottom', color='black', fontsize=64)
        
        axs[row, 0].text(-0.15, 0.1, str(row+1), transform=axs[row, 0].transAxes,
                     ha='left', va='center', color='black', fontsize=64, rotation=0)
        axs[4, 0].text(-0.35, 0.05, str('Row'), transform=axs[4, 0].transAxes,
                     ha='left', va='center', color='black', fontsize=64, rotation=90)
        
    
        for j in range(3):
            axs[row,j].set_xticks([])
            axs[row,j].set_yticks([])
        
        row += 1
        
for col in range(3):
    axs[0,col].set(xlim=(50,250), ylim=(250,100))  
    axs[1,col].set(xlim=(50,250), ylim=(260,110))  
    axs[2,col].set(xlim=(40,240), ylim=(260,110))    
    axs[3,col].set(xlim=(110,310), ylim=(275,125))  
    axs[4,col].set(xlim=(200,400), ylim=(310,160))  
    axs[4,0].set(xlim=(200,400), ylim=(260,110))    
    axs[5,col].set(xlim=(110,310), ylim=(280,130)) 
    axs[5,2].set(xlim=(110,310), ylim=(300,150)) 
    axs[6,col].set(xlim=(120,320), ylim=(240,90)) 
    axs[6,2].set(xlim=(120,320), ylim=(250,100)) 
    axs[7,col].set(xlim=(70,270), ylim=(310,160))   
    axs[8,col].set(xlim=(100,300), ylim=(310,160))       

In [ ]:
# eem

scale_dict = {0: 1, 1:1, 2:1,
             3:1, 4:1.5, 5:1.2,
             6:1, 7:1, 8:1,}

fig, axs= plt.subplots(ncols=3, nrows = 9, figsize=(21, 40))
fig.tight_layout()

row = 0

for i in [ 25, 29, 43, 64, 65, 67, 104,113, 121]:
        e1 = double.electrode1[i]
        e2 = double.electrode2[i]
        freq = double.freq[i]
        amp1 = double.amp1[i]
        subject = double.subject[i]
        single1 = single[(single.electrode1 == e1) & (single.freq == freq) &
                                (single.amp1 == amp1) & (single.subject == subject)]
        single2 = single[(single.electrode1 == e2) & (single.freq == freq) &
                                (single.amp1 == amp1) & (single.subject == subject)]
        
        new_height = int(round(single1.avg_image.iloc[0].shape[0]/scale_dict[row],0))
        new_width = int(round(single1.avg_image.iloc[0].shape[1]/scale_dict[row],0))
        axs[row,0].imshow(cv2.resize(single1.avg_image.iloc[0], (new_width, new_height)), cmap = 'gray_r')
        axs[row,1].imshow(cv2.resize(single2.avg_image.iloc[0], (new_width, new_height)), cmap = 'gray_r')       
        axs[row,2].imshow(cv2.resize(double.avg_image[i], (new_width, new_height)), cmap = 'gray_r')
        
        axs[row, 0].text(0.98, 0.02, str(e1), transform=axs[row, 0].transAxes,
                     ha='right', va='bottom', color='black', fontsize=64)
        axs[row, 1].text(0.98, 0.02, str(e2), transform=axs[row, 1].transAxes,
                         ha='right', va='bottom', color='black', fontsize=64)
        axs[row, 2].text(0.98, 0.02, str(e1) + '+' + str(e2), transform=axs[row, 2].transAxes,
                         ha='right', va='bottom', color='black', fontsize=64)
        
        axs[row, 2].text(1.2, 0.5, str(subject), transform=axs[row, 2].transAxes,
                     ha='right', va='center', color='black', fontsize=64, rotation=-90)
        axs[4, 2].text(1.4, 0.05, str('Subject'), transform=axs[4, 2].transAxes,
                     ha='right', va='center', color='black', fontsize=64, rotation=-90)
    
        for j in range(3):
            axs[row,j].set_xticks([])
            axs[row,j].set_yticks([])
        
        row += 1
        
for col in range(3):
    axs[0,col].set(xlim=(170,370), ylim=(310,160))  
    axs[1,col].set(xlim=(150,350), ylim=(330,180))  
    axs[2,col].set(xlim=(100,300), ylim=(280,130)) 
    axs[2,2].set(xlim=(100,300), ylim=(330,180))    
    axs[3,col].set(xlim=(150,350), ylim=(280,130))  
    axs[4,col].set(xlim=(30,230), ylim=(240,90))  
    axs[5,col].set(xlim=(90,290), ylim=(260,110)) 
    axs[6,col].set(xlim=(100,300), ylim=(240,90)) 
    axs[7,col].set(xlim=(160,360), ylim=(310,160))   
    axs[8,col].set(xlim=(120,320), ylim=(250,100))       